## VASP stresses

In [1]:
import numpy as np
from pymatgen.io.vasp.outputs import Outcar

In [3]:
def stresses_from_outcar( filename='OUTCAR' ):
    """Finds and returns stress tensors from the OUTCAR file.
      
    Args:
        filename (:obj:'str', optional): the name of the ``OUTCAR`` file to be read. Default is `OUTCAR`.
    Returns:
        (np.array): The force as found in the ``OUTCAR`` file, as a 1D numpy array.
    """
    outcar = Outcar(filename)
    stresses = outcar.read_table_pattern(
        header_pattern=r"\s+Fock(\s+[+-]?\d+\.\d+)*\n\s+-+",
        row_pattern=r"\s+\D+\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)",
        footer_pattern=r"\s+in kB.+",
        postprocess=lambda x: float(x),
        last_one_only=False
    )
    return np.array(stresses).flatten()

In [4]:
# outcar = Outcar('outcars/thermos/OUTCAR1')
stresses = stresses_from_outcar( filename='outcars/thermos/OUTCAR1' )

In [5]:
stresses # eV/(cell volume) XX YY ZZ XY YZ ZX

array([ 8.917071e+01,  1.545475e+01,  4.985392e+01,  5.278070e+00,
       -3.239010e+00, -1.571000e-02])

## LAMMPS stresses

### Code to run single step and pull out stresses on atoms.
Units = pressure\*volume units (bars*ang^3)
format = XX YY ZZ XY XZ YZ 

In [8]:
instance.command('compute stresses all stress/atom NULL') #requires NULL pair?
instance.command('dump stress all custom 1 stresses.dump c_stresses[*]')
instance.run(0)

### Reads in file adn skips the header, returning a (n,6) numpy array

In [37]:
ip_stresses = np.genfromtxt('stresses.dump', skip_header=9)